In [1]:
!pip install pulp

In [2]:
import numpy as np
import pandas as pd
from pulp import *

In [3]:
listSolvers(onlyAvailable=True)

['PULP_CBC_CMD', 'PULP_CHOCO_CMD']

In [34]:
def clean_strg(A):
  low_value = np.max(np.min(A, axis=1))
  high_value = np.min(np.max(A, axis=0))
  print(low_value, ' ', high_value)
  if np.isclose(low_value, high_value):
    game_price = low_value
  else:
    game_price = None
  return game_price

In [36]:
if clean_strg(matrix):
  print('Игра решается в чистых стратегиях. Цена игры:', clean_strg(matrix))
  strategy_first = np.argmax(np.min(matrix, axis=1))
  strategy_second = np.argmin(np.max(matrix, axis=0))
  print('Стратегия для первого игрока: ', strategy_first+1, '. Для второго: ', strategy_second+1)
else: 
  print('Игра не решается в чистых стратегиях.')
  if (matrix <= 0).any():
    a = np.abs(matrix.min()) + 1
    matrix += a
  else:
    a = 0

  LP1 = pulp.LpProblem('1player', 1)
  vars1 = LpVariable.dicts('1p', range(0, matrix.T.shape[1]), cat='Continuous', lowBound=0)
  LP1 += lpSum([vars1[var] for var in vars1]), "P1"
  for i in range(0, matrix.T.shape[0]):
    LP1+= lpSum([matrix.T[i][j]*vars1[j] for j in range(0, matrix.T.shape[1])]) >= 1
  print(LP1)
  slvr = PULP_CBC_CMD()
  s1 = LP1.solve(slvr)
  #print(LpStatus[s1])
  U = np.array([])
  for u in LP1.variables():
    if u.varValue != None:
      U = np.append(U, u.varValue)
    #print(u.name, '=', u.varValue)

  LP2 = pulp.LpProblem('2player', -1)
  vars2 = LpVariable.dicts('2p', range(0, matrix.shape[1]), cat='Continuous', lowBound=0)
  LP2 += lpSum([vars2[var] for var in vars2]), "P2"
  for i in range(0, matrix.shape[0]):
    LP2+= lpSum([matrix[i][j]*vars2[j] for j in range(0, matrix.shape[1])]) <= 1
  print(LP2)
  s2 = LP2.solve()
  V = np.array([])
  for v in LP2.variables():
    if v.varValue != None:
      V = np.append(V, v.varValue)
    #print(v.name, '=', v.varValue)

  I1 = 1 / sum(U)
  I2 = 1 / sum(V)
  p = np.round(U * I1, 2)
  q = np.round(V * I2, 2)
  I = np.round(np.mean([I1, I2]), 2) - a
  print('Игрок 1:')
  print(pd.DataFrame([p], columns=[f'Сратегия {i+1}' for i in range(len(p))],
                      index=['']).T, '\n')
  print(f'Игрок 2:')
  print(pd.DataFrame([q], columns=[f'Сратегия {i+1}' for i in range(len(q))],
                      index=['']).T, '\n')
  print(f'Цена игры: {I}')



2   2
2   2
Игра решается в чистых стратегиях. Цена игры: 2
Стратегия для первого игрока:  1 . Для второго:  1


In [21]:
matrix = np.array([[33, 10, 20, 26.5],
                   [50, 67, 11.5, 25], 
                   [23.5, 35, 40, 58.5]])



In [29]:
matrix = np.array([[-2, 6, 7, 8],
                   [-1, 3, -5, 0], 
                   [7, 5, 4, -3]])



In [32]:
matrix = np.array([[2, 4],
                   [1, -3]])

